In [3]:
#konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 3.8MB 45.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [1]:
import pandas as pd
import numpy as np

In [4]:
df_20 = pd.read_csv(r'C:\Users\haech\my_workspace\prac_scrapy\df_20_preprocessed.csv')

In [5]:
con_ls = list(df_20['content'].dropna())

In [6]:
len(con_ls)

38544

In [7]:
# Okt 토큰화
from konlpy.tag import Okt
okt = Okt()

okt_tokens = []
for c in con_ls:
    okt_token = okt.morphs(c)
    okt_tokens.append(okt_token)

In [8]:
len(okt_tokens)

38544

In [47]:
# Word2Vec embedding
from gensim.models import Word2Vec
embedding_model = Word2Vec(okt_tokens, size=100, window = 3, min_count=10, workers=4, iter=100, sg=1) # min_count=50, 50번 미만 등장 제외

In [61]:
# check embedding result
embedding_model.most_similar(positive=["댕댕이"], topn=20)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


KeyError: "word '댕댕이' not in vocabulary"

In [51]:
model_vec = embedding_model.wv

In [52]:
len(model_vec.vectors)

10699

In [53]:
from sklearn.manifold import TSNE

In [54]:
tsne = TSNE(random_state=0)
tsne_points = tsne.fit_transform(model_vec.vectors)

In [55]:
tsne_df = pd.DataFrame(tsne_points, columns=['x_coord', 'y_coord'])
tsne_df['word'] = [model_vec.index2word[i] for i in tsne_df.index]

In [56]:
tsne_df

,x_coord,y_coord,word
0,-3.175542,2.677223,이
1,-3.729389,0.106401,에
2,-9.573195,2.285562,을
3,-4.632515,0.254868,가
4,-9.900450,1.380208,의
...,...,...,...
10694,-7.879301,0.084960,제작자
10695,-34.713440,-3.451032,사군자
10696,-30.432993,-16.029337,오더대로
10697,-24.425657,-17.320423,입량


In [29]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [30]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 1000,
                   plot_height =1000,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='red', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show time!
show(tsne_plot);